In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
openai = OpenAI()
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
def get_page_source(url):
    response = requests.get(url)
    response.raise_for_status()  # Hata varsa bildirir
    return response.text  # Ham HTML metni döner

system_prompt = "You are an assistant analyzing the source of a website and checking for security vulnerabilities."

def user_prompt_for(url):
    user_prompt = "Below is the HTML source of the website:\n\n"
    user_prompt += get_page_source(url)  
    user_prompt += "\n\nPlease check this website and search for security vulnerabilities. "
    user_prompt += "If you don't find any, print 'No vulnerability found.' "
    user_prompt += "If you find a potential vulnerability risk, describe the vulnerability risk and print 'Potential Vulnerability Risk'."
    user_prompt += "If you find a direct, explicit vulnerability, describe the vulnerability and CVSS Score print 'ATTENTION! Vulnerability is Found.'"
    user_prompt += "If you find both a potential vulnerability risk and a direct, explicit vulnerability, describe them and CVSS Score print 'ATTENTION! Potential Vulnerability Risk and Direct Vulnerability are Found!!'"
    return user_prompt

def messages_for(url):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(url)}
    ]

def check_vuln(url):
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(url)
    )
    return response.choices[0].message.content



In [ ]:
def display_vuln(url):
    display_vuln = check_vuln(url)
    display(Markdown(display_vuln))

In [ ]:
display_vuln("https://edwarddonner.com")

In [ ]:
display_vuln("http://192.168.1.113/") #local apache server IP, contains xss_vulnerable_example.html

In [ ]:
display_vuln("https://www.google.com")